In [1]:
import numpy as np
import tensorflow as tf
from PIL import Image
import time
import cv2
import matplotlib.pyplot as plt

class Object_Detector():
    def top(self, top_left1 = 0): 
         self.top_left1 = top_left1 
      
    # getter method 
    def getTop(self): 
        return self.top_left1 
      
    # setter method 
    def setTop(self, x): 
        self.top_left1 = x         
        
    def bottom(self, bottom_right1 = 0): 
         self.bottom_right1 = bottom_right1
      
    # getter method 
    def getBottom(self): 
        return self.bottom_right1 
      
    # setter method 
    def setBottom(self, x): 
        self.bottom_right1 = x    
    
    def __init__(self, model_path):
        self.__load_model(model_path)
        print('model loaded')

    def __load_model(self, model_path):
        self.detection_graph = tf.Graph()
        with self.detection_graph.as_default():
            od_graph_def = tf.GraphDef()
            with tf.gfile.GFile(model_path, 'rb') as fid:
                serialized_graph = fid.read()
                od_graph_def.ParseFromString(serialized_graph)
                tf.import_graph_def(od_graph_def, name='')

        config = tf.ConfigProto()
        config.gpu_options.allow_growth= True

        with self.detection_graph.as_default():
            self.sess = tf.Session(config=config, graph=self.detection_graph)
            self.image_tensor = self.detection_graph.get_tensor_by_name('image_tensor:0')
            self.detection_boxes = self.detection_graph.get_tensor_by_name('detection_boxes:0')
            self.detection_scores = self.detection_graph.get_tensor_by_name('detection_scores:0')
            self.detection_classes = self.detection_graph.get_tensor_by_name('detection_classes:0')
            self.num_detections = self.detection_graph.get_tensor_by_name('num_detections:0')

        # load label_dict
        self.label_dict = {1: 'fish'}
        
        # warmup
        self.detect_image(np.ones((600, 600, 3)))
 

    def detect_image(self, image_np, score_thr=0.5, print_time=False):
        image_w, image_h = image_np.shape[1], image_np.shape[0]
    
        # Actual detection.
        t = time.time()
        (boxes, scores, classes, num) = self.sess.run(
          [self.detection_boxes, self.detection_scores, self.detection_classes, self.num_detections],
          feed_dict={self.image_tensor: np.expand_dims(image_np, axis=0)})
        if print_time:
            print('detection time :', time.time()-t)
        # Visualization of the results of a detection.
        for i, box in enumerate(boxes[scores>score_thr]):
            top_left = (int(box[1]*image_w), int(box[0]*image_h))
            self.setTop(top_left)
            bottom_right = (int(box[3]*image_w), int(box[2]*image_h))
            self.setBottom(bottom_right)
            cv2.rectangle(image_np, top_left, bottom_right, (0,255,0), 3)
            cv2.putText(image_np, self.label_dict[int(classes[0,i])], top_left, cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
    
        return image_np

In [2]:
# MODEL_PATH = 'fish_ssd_fpn_graph/frozen_inference_graph.pb'
MODEL_PATH = 'frozen_inference_graph.pb'
object_detector = Object_Detector(MODEL_PATH)

model loaded


In [ ]:
pic = "C:\\Users\\joshc\\OneDrive\\Desktop\\test\\fishs82.png"
img = cv2.imread(pic)
# img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

image = Image.open(pic)
img_ = object_detector.detect_image(img, score_thr=0.9)
# box = object_detector.box_cord(img, score_thr=0.99)
x,y = object_detector.getTop()
z, w =object_detector.getBottom()
croppedImage = image.crop((x,y,z,w))
croppedImage.show()
cv2.imshow("image", img_)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [12]:
import glob
import os
list = []
for filepath in glob.iglob( r"C:\\Users\\joshc\\Downloads\\preproc\\channelcatpre\\*.jpg"):
    list.append(filepath)
print(len(list))

1020


In [13]:
path2 = "C:\\Users\\joshc\\Downloads\\preproc\\channelCatfish"
j = 0
for i in list:
   
    img = cv2.imread(i)
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    image = Image.open(i)
    img_ = object_detector.detect_image(img, score_thr=0.9)
    # box = object_detector.box_cord(img, score_thr=0.99)
    x,y = object_detector.getTop()
    z, w =object_detector.getBottom()
    croppedImage = image.crop((x,y,z,w))
    croppedImage.save(os.path.join(path2, "image" + str(j) + ".png"))
    j+=1
